In [1]:
import os
import argparse
import time

import matplotlib as mpl
mpl.use('TkAgg') # sets the backend for matplotlib
import matplotlib.pyplot as plt

from utils.redraw_obstacles import redraw_obstacles, set_non_blocking
from utils.create_draw_params import create_draw_params
from commonroad.common.file_reader import CommonRoadFileReader
from commonroad.visualization.draw_dispatch_cr import draw_object

import numpy as np

from commonroad.prediction.prediction import Prediction, Occupancy
def occupancy_at_time(self, time_step: int):
    """
    Returns the predicted occupancy of the obstacle at a specific time step.

    :param time_step: discrete time step
    :return: predicted occupancy of the obstacle at time step
    """
    occupancy = None
    if time_step == self.initial_state.time_step:
        shape = self.obstacle_shape.rotate_translate_local(
            self.initial_state.position, self.initial_state.orientation)
        occupancy = Occupancy(time_step, shape)
    elif time_step > self.initial_state.time_step and time_step < (self.initial_state.time_step + len(self._prediction.trajectory.state_list)) and   self._prediction is not None:
        occupancy = self._prediction.occupancy_at_time_step(time_step)
    return occupancy

from commonroad.scenario.obstacle import StaticObstacle, DynamicObstacle, Obstacle
DynamicObstacle.occupancy_at_time = occupancy_at_time

In [2]:
file_path = '/home/rong/VAE-Motion-Planning/scenarios/commonroad_data/highway-ramp.cr.xml'
scenario, _ = CommonRoadFileReader(file_path).open()

In [3]:
def show_lane(scenario):
    plt.figure(figsize=(250, 100))
    draw_params={'time_begin':0}
    handles = {}
    draw_object(scenario, handles=handles, draw_params=draw_params)
    plt.gca().autoscale()
    plt.gca().set_aspect('equal')
    plt.show()

In [4]:
show_lane(scenario)

KeyboardInterrupt: 

In [64]:
def plt_window(sce, dur, draw_params, size, pos):
    set_non_blocking()  # ensures interactive plotting is activated
    figsize = size
    fig = plt.figure(figsize=(figsize[0] / inch_in_cm, figsize[1] / inch_in_cm))
#     ax = plt.Axes(fig, [0., 0., 1., 1.])
#     ax.set_axis_off()
    # ax.set_facecolor("black")
    # ax.set_cmap("binary")
#     fig.add_axes(ax)
    fig.patch.set_facecolor('black')
    plt.xlim(pos[0] - size[0]/2, pos[0] + size[0]/2)
    plt.ylim( pos[1] -size[1]/2, pos[1] + size[1]/2)
#     plt.set_cmap("plasma")
    plt.gca().set_aspect('equal')
    handles = {}  # collects handles of obstacle patches, plotted by matplotlib
    plot_limits = [pos[0] - size[0]/2, pos[0] + size[0]/2, pos[1] +-size[1]/2, pos[1] + size[1]/2]
    draw_params['time_begin'] = dur[0]
    draw_object(sce, handles=handles, draw_params=draw_params, plot_limits=plot_limits)
    # plt.gray()
    fig.canvas.draw()
    for handles_i in handles:
        if not handles_i:
            handles.pop()

    data = np.zeros(( (duration[1] - duration[0],) + fig.canvas.get_width_height()[::-1]), dtype='bool')

    t1 = time.time()
    # loop where obstacle positions are updated
    for i in range(dur[0], dur[1]):
        # ...
        # change positions of obstacles
        # ...
        draw_params['time_begin'] = i
        redraw_obstacles(sce, handles=handles, figure_handle=fig, plot_limits=plot_limits, draw_params=draw_params)
        buffer = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (4,))
        buffer = buffer[:, : , 0].astype(bool)
        data[(i-dur[0]), :, : ] = buffer
        print('fps:', (i - dur[0])/(time.time()-t1))
    return data

In [65]:
size = [200, 40]
pos = [350, 135]
duration = [0, 100]
inch_in_cm = 10
draw_params = create_draw_params()

origin_data = plt_window(sce=scenario, dur=duration, draw_params=draw_params, size=size, pos=pos)

fps: 0.0
fps: 5.923547430843995
fps: 8.165780190170624
fps: 9.28152813245467
fps: 9.85945651158267
fps: 10.348987777532136
fps: 10.508861981589527
fps: 10.62683887668722
fps: 10.804120285654763
fps: 10.988313573597823
fps: 11.10208385886182
fps: 11.20985279843335
fps: 11.174611970807977
fps: 11.145036033454351
fps: 11.216494593298785
fps: 11.464420233207782
fps: 11.571924322368817
fps: 11.634117548968403
fps: 11.679312769319662
fps: 11.668801433316826
fps: 11.683729274130592
fps: 11.669947036319371
fps: 11.7041960715372
fps: 11.718840611047895
fps: 11.737045162125309
fps: 11.816037905941915
fps: 11.768484021256931
fps: 11.749730758743649
fps: 11.733261292915035
fps: 11.747828167265528
fps: 11.692352295092022
fps: 11.683743185111913
fps: 11.692717975702047
fps: 11.66442544888965
fps: 11.6304028711628
fps: 11.62821740996369
fps: 11.601460357750307
fps: 11.602440838217799
fps: 11.612571344720802
fps: 11.570915297710226
fps: 11.617515888966036
fps: 11.633025266378892
fps: 11.60935482161766

TclError: invalid command name "pyimage360"

In [11]:
def plt_data(observe):
    fig = plt.figure()
    plt.gray()
    if len(observe.shape) <=2:
        plt.imshow(observe)
    elif len(observe.shape) > 3:
        for r in observe:
            myobj = plt.imshow(r[0, :, :])
            for rr in r:
                myobj.set_data(rr)
                plt.draw()
                plt.pause(0.01)
    else:
        myobj = plt.imshow(observe[0, :, :])
        for row in observe:
            myobj.set_data(row)
            plt.draw()
            plt.pause(0.01)

In [12]:
from commonroad.scenario.scenario import Scenario

ego_scenario = Scenario(dt=0.1, benchmark_id=scenario.benchmark_id)
ego_scenario.lanelet_network = scenario.lanelet_network

In [96]:
def theTrueData(leng, start_egoid, dt):
    #  初始化fig
    set_non_blocking()  # ensures interactive plotting is activated
    size = [400, 30]
    pos = [350, 135]
    inch_in_cm = 10
    fig = plt.figure(figsize=(size[0] / inch_in_cm, size[1] / inch_in_cm))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    fig.patch.set_facecolor('black')
    plt.xlim(pos[0] - size[0]/2, pos[0] + size[0]/2)
    plt.ylim( pos[1] -size[1]/2, pos[1] + size[1]/2)
    plt.set_cmap("plasma")
    plt.gca().set_aspect('equal')

    #     draw scenario
    handles = {}  # collects handles of obstacle patches, plotted by matplotlib
    draw_params = create_draw_params()
    draw_object(scenario.lanelet_network, handles=handles, draw_params=draw_params)
    fig.canvas.draw()
    for handles_i in handles:
        if not handles_i:
            handles.pop()

    #   初始化data
    start_goal = np.zeros((leng, 8), dtype=np.float64)
    observed = np.zeros(( (leng, dt,) + fig.canvas.get_width_height()[::-1]), dtype=np.bool)
    data = np.zeros((leng,  dt, 4), dtype=np.float64)

    #   从哪辆车开始呢
    ego_id = start_egoid
    index = 0
    while True:
        ego = None
        while(scenario.obstacle_by_id(ego_id) == None):
            ego_id = ego_id + 1
        ego = scenario.obstacle_by_id(ego_id)
        ego_id = ego_id + 1
        print("ego_id: ", ego_id)
    #   获得那辆车的traj，initial_time, terminated_time
        scenario.remove_obstacle(ego)
        traj = ego.prediction.trajectory
        initial_time = traj.initial_time_step
        terminated_time = traj.state_list[-1].time_step
    #  开始循环
        start_time = initial_time + 2* dt
        end_time = start_time + 2 * dt

        while end_time < terminated_time - 2* dt:
            t =int( (start_time - initial_time)/2)
            pos = traj.state_list[t].position + [100, 0]
    #   实际traj数据
            for i in range(0, dt):
                data[index, i, :] = [traj.state_list[t+i].position[0], traj.state_list[t+i].position[1], traj.state_list[t+i].orientation, traj.state_list[t+i].velocity]
    #   起点，终点
            start_goal[index, 0:4] = data[index, 0, :]
            start_goal[index, 4:8] = data[index, -1, :]

            plot_limits = [start_goal[index, 0] - size[0]/2 + 50, start_goal[index, 0] + size[0]/2 + 50, start_goal[index, 1]+-size[1]/2,start_goal[index, 1]+ size[1]/2]
#             print("start_goal: ", start_goal[index, :])
            t1 = time.time()
            # loop where obstacle positions are updated
            for i in range(0, dt):
                # ...
                # change positions of obstacles
                # ...
                draw_params['time_begin'] = start_time + 2*i
                redraw_obstacles(scenario, handles=handles, figure_handle=fig, plot_limits=plot_limits, draw_params=draw_params)
#                 尝试查看ego轨迹
#                 plt.plot(data[index, :, 0], data[index, :, 1])
                buffer = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (4,))
                buffer = buffer[:, : , 0].astype(bool)
    #   observed data
                observed[index, i, :, : ] = buffer
    #                 print('fps:', i/(time.time()-t1), " time:", i)
            start_time = start_time + 2 * dt
            end_time = start_time + 2 * dt
            
            index = index + 1
            if(index == leng): break
        scenario.add_objects(ego)
        if(index == leng): break
    plt.close(fig)
    return start_goal, observed, data, ego_id + 1

In [97]:
import torch

leng = 100
dt = 25
start_ego = 0
last_ego = start_ego

i_data = 0 
for i_data in range(0, 1000):
    start_goal_i, observed_i, data_i, last_ego =  theTrueData(leng, last_ego, dt)
    PATH = '/home/rong/disk/mydata/data_' + str(i_data);
    torch.save({
                'index': i_data,
                'start_goal': start_goal_i,
                'observation': observed_i,
                'data': data_i
                }, PATH)

/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/commonroad/scenario/scenario.py:183: UserWarning: <Scenario/obstacle_by_id> Obstacle with ID 127 is not contained in the scenario.
  % obstacle_id)


ego_id:  131
ego_id:  132
ego_id:  133
ego_id:  134
ego_id:  135
ego_id:  137
ego_id:  138
ego_id:  139
ego_id:  140
ego_id:  141
ego_id:  143
ego_id:  144
ego_id:  145
ego_id:  146
ego_id:  147
ego_id:  149
ego_id:  150
ego_id:  151
ego_id:  152
ego_id:  153
ego_id:  155
ego_id:  156
ego_id:  157
ego_id:  158
ego_id:  159
ego_id:  161
ego_id:  162
ego_id:  163
ego_id:  164
ego_id:  165
ego_id:  167
ego_id:  168
ego_id:  169
ego_id:  170
ego_id:  171
ego_id:  173
ego_id:  174
ego_id:  175
ego_id:  176
ego_id:  177
ego_id:  179
ego_id:  180
ego_id:  181
ego_id:  182
ego_id:  183
ego_id:  185
ego_id:  186
ego_id:  187
ego_id:  188
ego_id:  189
ego_id:  191
ego_id:  192
ego_id:  193
ego_id:  194
ego_id:  195
ego_id:  196
ego_id:  198
ego_id:  199
ego_id:  200
ego_id:  201
ego_id:  202
ego_id:  204
ego_id:  205
ego_id:  206
ego_id:  207
ego_id:  208
ego_id:  210
ego_id:  211
ego_id:  212
ego_id:  213
ego_id:  214
ego_id:  216
ego_id:  217
ego_id:  218
ego_id:  219
ego_id:  220
ego_id:  222

TclError: invalid command name "pyimage952"

In [98]:
for i_data in range(39, 1000):
    start_goal_i, observed_i, data_i, last_ego =  theTrueData(leng, last_ego, dt)
    PATH = '/home/rong/disk/mydata/data_' + str(i_data);
    torch.save({
                'index': i_data,
                'start_goal': start_goal_i,
                'observation': observed_i,
                'data': data_i
                }, PATH)

/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/commonroad/scenario/scenario.py:183: UserWarning: <Scenario/obstacle_by_id> Obstacle with ID 369 is not contained in the scenario.
  % obstacle_id)


ego_id:  371
ego_id:  372
ego_id:  373
ego_id:  374
ego_id:  375
ego_id:  377
ego_id:  378
ego_id:  379
ego_id:  380
ego_id:  381
ego_id:  383
ego_id:  384
ego_id:  385
ego_id:  386
ego_id:  387
ego_id:  389
ego_id:  390
ego_id:  391
ego_id:  392
ego_id:  393
ego_id:  394
ego_id:  396
ego_id:  397
ego_id:  398
ego_id:  399
ego_id:  400
ego_id:  402
ego_id:  403
ego_id:  404
ego_id:  405
ego_id:  406
ego_id:  408
ego_id:  409
ego_id:  410
ego_id:  411
ego_id:  412
ego_id:  414
ego_id:  415
ego_id:  416
ego_id:  417
ego_id:  418
ego_id:  420
ego_id:  421
ego_id:  422
ego_id:  423
ego_id:  424
ego_id:  426
ego_id:  427
ego_id:  428
ego_id:  429
ego_id:  430
ego_id:  432
ego_id:  433
ego_id:  434
ego_id:  435
ego_id:  436
ego_id:  438
ego_id:  439
ego_id:  440
ego_id:  441
ego_id:  443
ego_id:  444
ego_id:  445
ego_id:  446
ego_id:  447
ego_id:  449
ego_id:  450
ego_id:  451
ego_id:  452
ego_id:  453
ego_id:  455
ego_id:  456
ego_id:  457
ego_id:  458
ego_id:  459
ego_id:  461
ego_id:  462

MemoryError: 

In [9]:
import torch
PATH = 'data/data_' + str(i_data);
torch.save({
            'index': i_data,
            'start_goal': start_goal_i,
            'observation': observed_i,
            'data': data_i
            }, PATH)

In [12]:
plt_data(observed_i)

In [26]:
load_data = torch.load(PATH)
load_start_goal = load_data['start_goal']
load_observed = load_data["observation"]
load_data = load_data["data"]

In [14]:
plt_data(load_observed)

Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/tkinter/__init__.py", line 749, in callit
    func(*args)
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/matplotlib/backends/_backend_tk.py", line 346, in idle_draw
    self.draw()
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/matplotlib/backends/backend_tkagg.py", line 9, in draw
    super(FigureCanvasTkAgg, self).draw()
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/matplotlib/backends/backend_agg.py", line 402, in draw
    self.figure.draw(self.renderer)
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/matplotlib/artist.py", line 50, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/

In [86]:
a = Range(4)

NameError: name 'Range' is not defined

In [89]:
for x in range(0, 3):
    print(x)

0
1
2
